In [40]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse

from functools import reduce
from collections import namedtuple
from copy import copy
from tqdm import tqdm

In [41]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [42]:
num_points = num_sets = 5000
density = .3

x = make_set_covering_problem(num_points, num_sets, density)
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: True


In [43]:
State = namedtuple("State", ["taken", "not_taken"])


def goal_check(state):
    return np.all(
        reduce(
            np.logical_or,
            [x.getrow(i).tocsr().toarray() for i in state.taken],
            [False for _ in range(num_points)],
        )
    )


In [44]:
assert goal_check(State(set(range(num_sets)), set())), "Problem not solvable"

In [45]:
def random_tweak(state):
    index = randint(0, num_points-1)
    new_state = State(state.taken ^ {index}, state.not_taken ^ {index})
    return new_state


def evaluate1(state):
    return np.sum(
        reduce(
            np.logical_or,
            [x.getrow(i).tocsr().toarray() for i in state.taken],
            [False for _ in range(num_points)],
        )
    ), -len(state.taken)


In [46]:
def covered(state):
    return reduce(
        np.logical_or,
        (x.getrow(i).tocsr().toarray() for i in state.taken),
        [False for _ in range(num_points)],
    )


def max_cover_tweak(state):
    cover = covered(state)

    _, best = max(
        (
            (np.sum(np.logical_and(x.getrow(i).tocsr().toarray(), np.logical_not(cover))), i)
            for i in state.not_taken
        )
    )

    new_state = State(state.taken ^ {best}, state.not_taken ^ {best})

    return new_state


`evaluate2` resulted to be too slow for this kind of approach

In [47]:
def max_left(state):    
    _, res = max(
        (np.sum(
            np.logical_and(x.getrow(i).tocsr().toarray(), np.logical_not(covered(state)))
        ), i) for i in state.not_taken
    )
    return res

def evaluate2(state):
    starting_steps = len(state.taken)
    iter_state = copy(state)
    
    while not goal_check(iter_state):
        max = max_left(iter_state)
        iter_state = State(iter_state.taken ^ {max},
                       iter_state.not_taken ^ {max})
    
    return np.sum(covered(state)), -len(state), -(len(state.taken) - starting_steps)

In [48]:
def run_alg(tweak, evaluate):
    print(tweak.__name__, evaluate.__name__)
    current_state = State(set(), set(range(num_points)))
    
    with tqdm(total=None) as pbar:
        while not goal_check(current_state):
            new_state = tweak(current_state)
            if evaluate(new_state) > evaluate(current_state):
                current_state = new_state
            pbar.update(1)
        
    print(current_state.taken)

In [50]:
run_alg(random_tweak, evaluate1)
run_alg(max_cover_tweak, evaluate1)
# run_alg(max_cover_tweak, evaluate2)

random_tweak evaluate1


23it [00:00, 91.87it/s] 


{770, 1163, 1819, 2972, 2078, 2338, 1058, 4901, 38, 1705, 815, 1093, 3783, 1232, 3024, 3165, 1001, 4078, 2420, 1654, 2942}
max_cover_tweak evaluate1


13it [00:19,  1.54s/it]

{4866, 1987, 1090, 1386, 3531, 3534, 48, 4593, 2611, 3860, 1075, 1688, 2206}
